In [2]:
import sqlite3
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

conn = sqlite3.connect('../DB/unknome_db_03_Nov_2023.sqlite')

In [3]:
data_interes = ['accessions',
                'subcell_locs',
                'species',
                'pfam_ids',
                'knownness']

select = ''
for i in data_interes:
    select += i + ", "
select = select[:len(select)-2]


consulta  = f"SELECT {select} FROM Protein WHERE knownness is not null"

df = pd.read_sql_query(consulta, conn)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147942 entries, 0 to 147941
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   accessions    147942 non-null  object 
 1   subcell_locs  92377 non-null   object 
 2   species       147942 non-null  object 
 3   pfam_ids      140705 non-null  object 
 4   knownness     147942 non-null  float64
dtypes: float64(1), object(4)
memory usage: 5.6+ MB


In [4]:
df_pfam = df[["accessions", "pfam_ids", "knownness"]].copy()
df_pfam.dropna(inplace=True)
df_pfam['pfam_ids'] = df_pfam["pfam_ids"].str.split(";")

In [5]:
df_pfam

,accessions,pfam_ids,knownness
0,F4HZG9;F4HZH0;Q8GYQ0;Q9ZPZ5;W6HYK5,"[PF05195, PF00557]",4.0
1,Q1PDV6;A0MFG6,[PF00069],6.4
2,P56757,"[PF00006, PF00306, PF02874]",2.1
3,O04567;Q0WPX0,"[PF13516, PF08263, PF07714]",0.0
4,Q0WRJ7;Q9M222,[PF00254],2.4
...,...,...,...
147937,Q9USL1,"[PF08662, PF00400]",1.5
147938,O42653,[PF03109],0.5
147939,Q9HDX3,[PF07859],0.0
147940,Q09839,[PF00294],0.5


In [6]:
df_pfam["pfam_ids"] = df_pfam["pfam_ids"].str.join(";")
pfam_dummies = df_pfam["pfam_ids"].str.get_dummies(sep=";")

In [7]:
pfam_dummies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140705 entries, 0 to 147941
Columns: 8883 entries, PF00001 to PF20624
dtypes: int64(8883)
memory usage: 9.3 GB


In [8]:
for column in tqdm(pfam_dummies.columns, desc="Procesando columnas"):
    pfam_dummies[column] = pfam_dummies[column].astype(bool)

Procesando columnas: 100%|██████████| 8883/8883 [00:09<00:00, 972.94it/s] 


In [9]:
pfam_dummies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140705 entries, 0 to 147941
Columns: 8883 entries, PF00001 to PF20624
dtypes: bool(8883)
memory usage: 1.2 GB


In [10]:
df_pfam = pd.concat([df_pfam, pfam_dummies], axis=1)
del pfam_dummies
df_pfam.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140705 entries, 0 to 147941
Columns: 8886 entries, accessions to PF20624
dtypes: bool(8883), float64(1), object(2)
memory usage: 1.2+ GB


In [12]:
df_pfam.drop(["pfam_ids", 'accessions'], axis=1, inplace=True)
df_pfam.head(1)

df_pfam.to_csv("df_pfam.csv", index=False)